# Otimização com Gradiente Descendente

Utilizaremos o **Gradiente Descendente** para encontrar os parâmetros ideais (`intercept` e `slope`) da nossa Reta de Regressão:

$$
y_i = \beta_0 + \beta_1 x_i + \epsilon_i
$$

**Onde:**
* **$y_i$**: Variável dependente (resposta) para a observação $i$.
* **$x_i$**: Variável independente (explicativa) para a observação $i$.
* **$\beta_0$**: **Intercept** (parâmetro que vamos otimizar).
* **$\beta_1$**: **slope** (parâmetro que vamos otimizar).
* **$\epsilon_i$**: **Erro aleatório**, a diferença entre o modelo e a realidade.

O objetivo é ajustar $\beta_0$ e $\beta_1$ simultaneamente para minimizar o erro entre as previsões da reta e os dados reais (`x_data`, `y_data`).

##Bibliotecas que usaremos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

## Definição das variaveis com os dados que usaremos:

In [ ]:
import numpy as np
intercept = 0
slope = 1
x_data = np.array([0.5, 2.3, 2.9])
y_data = np.array([1.4, 1.9, 3.2])
max_iterations = 1000
precision = 0.0001
learning_rate = 0.01
list_intercepts = []
list_slopes = []
list_sum_of_squared_residuals = []

##definição de funcões
Para facilitar a leitura, vamos associar as variáveis do código aos símbolos matemáticos:

> Adicionar aspas



* **$b$**: `current_intercept`
* **$m$**: `current_slop`
* **$x_i$**: `x_data`
* **$y_i$**: `y_data`

### 1. Derivada em relação ao Intercepto ($b$)
Correspondente à função `calculate_derivative_intercept`. Esta equação nos diz como o erro muda se ajustarmos apenas a altura da linha.

$$
\frac{\partial SSR}{\partial b} = \sum_{i=1}^{n} -2 (y_i - (b + m x_i))
$$

In [ ]:
def calculate_derivative_intercept(current_intercept, current_slop):
    predictions = current_intercept + current_slop * x_data
    residuals = y_data - predictions
    return np.sum(-2 * residuals)

### 2. Derivada em relação à Inclinação ($m$)
Correspondente à função `calculate_derivative_slop`. Esta equação nos diz como o erro muda se ajustarmos a angulação da linha. Note que aqui multiplicamos por $x_i$ no final (Regra da Cadeia).

$$
\frac{\partial SSR}{\partial m} = \sum_{i=1}^{n} -2 (y_i - (b + m x_i)) \cdot x_i
$$

In [ ]:
def calculate_derivative_slop(current_intercept, current_slop):
    predictions = current_intercept + current_slop * x_data
    residuals = y_data - predictions
    return np.sum(-2 * residuals * x_data)

### Executando o Gradiente Descendente (Otimização de Intercepto e Slope)

A cada iteração (passo do loop), o código realiza três tarefas principais, utilizando **todo o conjunto de dados**:

1.  **Calcula os Gradientes:** Determina a direção do erro para ambos os parâmetros com base em **todos os pontos** de dados simultaneamente.
2.  **Atualiza Intercepto e Slope:** Ajusta tanto a altura quanto a inclinação da reta para minimizar o erro.
3.  **Verifica a Precisão:** Se os passos de atualização para o intercepto e a inclinação forem pequenos demais (menores que a precisão definida), o loop encerra.

O processo se repete até que o gradiente seja próximo de zero (convergência) ou o maximo de iterações for atingido.

Também armazenamos o histórico dos valores de intercepto e inclinação para a visualização da convergência do algoritmo no gráfico posterior.

In [ ]:
for i in range(max_iterations):
    list_intercepts.append(intercept)
    list_slopes.append(slope)

    derivative_intercept = calculate_derivative_intercept(intercept, slope)
    derivative_slope = calculate_derivative_slop(intercept, slope)

    intercept_step_size = learning_rate * derivative_intercept
    slope_step_size = learning_rate * derivative_slope

    print(f"""Step: {i+1},
              Step Size Intercept: {intercept_step_size:.4f},
              Step Size Slope: {slope_step_size:.4f},
              Old intercept: {intercept:.4f},
              Old slope: {slope:.4f},
              New intercept: {(intercept - intercept_step_size):.4f},
              New slope: {(slope - slope_step_size):.4f}""",
              flush=True)

    intercept = intercept - intercept_step_size
    slope = slope - slope_step_size

    if np.abs(intercept_step_size) < precision and np.abs(slope_step_size) < precision:
        break

print(f"\n--- Treinamento Concluído ---")
print(f"Intercept Final: {intercept:.4f}")
print(f"Slope Final: {slope:.4f}")
print(f"Total de Iterações: {len(list_intercepts)}")

Step: 1,
              Step Size Intercept: -0.0160,
              Step Size Slope: -0.0080,
              Old intercept: 0.0000,
              Old slope: 1.0000,
              New intercept: 0.0160,
              New slope: 1.0080
Step: 2,
              Step Size Intercept: -0.0141,
              Step Size Slope: -0.0039,
              Old intercept: 0.0160,
              Old slope: 1.0080,
              New intercept: 0.0301,
              New slope: 1.0119
Step: 3,
              Step Size Intercept: -0.0128,
              Step Size Slope: -0.0012,
              Old intercept: 0.0301,
              Old slope: 1.0119,
              New intercept: 0.0430,
              New slope: 1.0132
Step: 4,
              Step Size Intercept: -0.0119,
              Step Size Slope: 0.0006,
              Old intercept: 0.0430,
              Old slope: 1.0132,
              New intercept: 0.0549,
              New slope: 1.0126
Step: 5,
              Step Size Intercept: -0.0113,
              Step S

### Animação da Convergência (Intercepto + Inclinação)

O código a seguir gera a visualização final do algoritmo de **Gradient Descent**, otimizando o intercept e o slope.

A função de atualização (`update`) reconstrói a equação da reta para cada iteração do histórico de treinamento:

$$\hat{y} = \text{slope}_{\text{atual}} \cdot x + \text{intercept}_{\text{atual}}$$

Diferente do exemplo anterior, tanto o slope quanto o intercept são alterados continuamente.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.scatter(x_data, y_data, color='blue', zorder=5, label='Pontos Originais')
ax.set_title('Gradiente Descendente: Convergência do Intercept e Slope')
ax.set_xlabel('height')
ax.set_ylabel('weight')
ax.legend()
ax.grid(True)

x_line_anim = np.linspace(min(x_data) - 0.5, max(x_data) + 0.5, 100)

line, = ax.plot(x_line_anim,
                list_slopes[0] * x_line_anim + list_intercepts[0],
                color='red',
                label='Linha de Regressão')
iteration_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, color='green')

def update(frame):
    current_intercept = list_intercepts[frame]
    current_slope = list_slopes[frame]
    y_line_anim = current_slope * x_line_anim + current_intercept
    line.set_ydata(y_line_anim)
    iteration_text.set_text(f'Iteração: {frame+1}\nIntercept: {current_intercept:.4f}\nSlope: {current_slope:.4f}')
    return line, iteration_text,

ani = FuncAnimation(fig, update, frames=len(list_intercepts), interval=20, blit=True)

plt.close(fig)
HTML(ani.to_jshtml())

Output hidden; open in https://colab.research.google.com to view.